In [13]:
from anchor import utils
from anchor import anchor_tabular
import pandas as pd
import sklearn.ensemble
import numpy as np
from anchor import anchor_tabular

In [14]:
dataset_folder = '../datasets/'
dataset = utils.load_dataset('adult', balance=True, dataset_folder=dataset_folder, discretize=True)
print(dataset.train[0])

[ 3.  6. 15.  2. 12.  0.  4.  1.  2.  0.  1. 39.]


In [15]:
c = sklearn.ensemble.RandomForestClassifier(n_estimators=50, n_jobs=5)
c.fit(dataset.train, dataset.labels_train)
print('Train', sklearn.metrics.accuracy_score(dataset.labels_train, c.predict(dataset.train)))
print('Test', sklearn.metrics.accuracy_score(dataset.labels_test, c.predict(dataset.test)))

Train 0.9350338780390594
Test 0.8489483747609943


In [16]:
transformed_data = []

for r in dataset.train:
    transformed_data.append([dataset.categorical_names[i][int(r_)] for i, r_ in enumerate(r)])        

print(transformed_data[0])

train_data = pd.DataFrame(transformed_data, columns=dataset.feature_names)
print(train_data.shape)
nb_vals = 25

print(dataset.train[0])
print(dataset.train[0].reshape(1, -1))
y=[]
for k in range(0, nb_vals):
    y.append(c.predict(dataset.train[k].reshape(1, -1))[0])


['Age > 48.00', 'Self-emp-not-inc', 'Some-college', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '2', '0', '40.00 < Hours per week <= 45.00', 'United-States']
(12545, 12)
[ 3.  6. 15.  2. 12.  0.  4.  1.  2.  0.  1. 39.]
[[ 3.  6. 15.  2. 12.  0.  4.  1.  2.  0.  1. 39.]]


In [20]:
import arg_explainer as ae
from importlib import reload
reload(ae)
import arg_explainer as ae
explainer = ae.ArgTabularExplainer(c, train_data.iloc[0:nb_vals], y, 'adult_' + str(nb_vals), compute=True)

Generating arguments
300  potential arg checked.
len  1 : 16 ,  15
1650  potential arg checked.
len  2 : 43 ,  66
5500  potential arg checked.
len  3 : 18 ,  9
12375  potential arg checked.
len  4 : 15 ,  0
19800  potential arg checked.
len  5 : 0 ,  0
Saving to  saves\adult_25_minimals.df


In [22]:
G = explainer.build_attack_graph(compute=True, display_graph=False)

def graph_to_tgf(G):
    tgf_file = 'tgf_test.tgf'
    node_dict = {}
    i_ = 0
    with open(tgf_file, 'w') as f:
        for n in G.nodes():
            i_ += 1
            node_dict[n] = i_
            f.write(str(i_)+ '\n')
        for u, v in G.edges():
            f.write(str(node_dict[u]) + ' ' + str(node_dict[v]) + '\n')
            f.write(str(node_dict[v]) + ' ' + str(node_dict[u]) + '\n')

    return node_dict
    
node_dict = graph_to_tgf(G)

92 90  args total
adult_25
len(R_atk) =  7093
5 highest degrees: [(frozenset({8, 58, 45}), 89) (frozenset({48, 58, 51, 6}), 89)
 (frozenset({50}), 90) (frozenset({47}), 92) (frozenset({46}), 92)]
5 lowest degrees: [(frozenset({0, 8, 58}), 58) (frozenset({0, 18}), 61)
 (frozenset({3, 4}), 62) (frozenset({0, 32, 58}), 63)
 (frozenset({0, 32, 45}), 63)]


In [54]:
from typing import OrderedDict
import networkx as nx
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
#print(explainer.categorical_names)
#print(explainer.categorical_features)
#print(explainer.ordinal_features)
#le.set_params(explainer.feature_names)

def generate_instance_random(constraints):
    instance = OrderedDict.fromkeys(explainer.dataset.columns)
    for f in constraints:
        col=explainer.col_p_feature[f]
        instance[col] = explainer.feature_names[f].split('_')[1]
        fill_instance(instance, 'random')
    return list(instance.values())

def fill_instance(instance, strategy):
    if strategy == 'random':
        # fill the instance with a random values sampled from baseline dataset
        for i_, col in enumerate(instance.keys()):
            if instance[col] is None:
                instance[col] = explainer.dataset[col].sample(1, random_state=1).values[0]
    elif strategy == 'most_frequent':
        # fill the instance with the most frequent value for each column
        for k, i_ in enumerate(instance.keys()):
            if instance[k] is None:
                instance[k] = explainer.dataset[i_].value_counts().index[0]
    
hist = nx.degree_histogram(G)

print('max degree:', max(hist))
max_deg_node = sorted(G.degree, key=lambda x: x[1], reverse=True)
attackers = G.edges(max_deg_node[0][0])
# print(attackers)

def instance2encoded(instance, dataset):
    encoded = []
    for col in dataset.categorical_features:
        encoded.append(dataset.categorical_names[col].index(instance[col]))
    return encoded

for a in attackers:
    print(a, [explainer.feature_names[k] for k in a[1]])
    inst = generate_instance_random(set.union(set(a[0]), set(a[1])))
    print(inst)
    print(instance2encoded(inst, dataset))

max degree: 15
(frozenset({56}), frozenset({25, 4})) ['Marital Status_Never-married', 'Workclass_?']
['Age > 48.00', '?', 'Bachelors', 'Never-married', 'Sales', 'Husband', 'White', 'Male', '2', '1', 'Hours per week > 45.00', 'United-States']
[3, 0, 9, 4, 12, 0, 4, 1, 2, 1, 2, 39]
(frozenset({56}), frozenset({7, 31})) ['Workclass_Private', 'Occupation_Farming-fishing']
['Age > 48.00', 'Private', 'Bachelors', 'Married-civ-spouse', 'Farming-fishing', 'Husband', 'White', 'Male', '2', '1', 'Hours per week > 45.00', 'United-States']
[3, 4, 9, 2, 5, 0, 4, 1, 2, 1, 2, 39]
(frozenset({56}), frozenset({16, 49, 59, 6})) ['Education_Bachelors', 'Race_White', 'Hours per week_Hours per week <= 40.00', 'Workclass_Local-gov']
['Age > 48.00', 'Local-gov', 'Bachelors', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '2', '1', 'Hours per week <= 40.00', 'United-States']
[3, 2, 9, 2, 12, 0, 4, 1, 2, 1, 0, 39]
(frozenset({56}), frozenset({52, 7, 30, 71})) ['Capital Gain_0', 'Workclass_Private', 

In [8]:
explainer.set_strategy('max_covc', 'existence')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.3927588898759051


KeyboardInterrupt: 

In [ ]:
explainer.set_strategy('max_covc', 'universal')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.40616693248272195
Number of extensions: 521076
len(max_cov_exts)= 521074 / 521076
Covc strategy's coverage: 2
Top 5 covs: [2, 2, 2, 2, 2]
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 0 Arg 1/0: None
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 0 Arg 1/0: None
id: 4 coverage: 0 Arg 1/0: None
id: 5 coverage: 0 Arg 1/0: None
id: 6 coverage: 0 Arg 1/0: None
id: 7 coverage: 0 Arg 1/0: None
id: 8 coverage: 0 Arg 1/0: None
id: 9 coverage: 0 Arg 1/0: None
id: 10 coverage: 0 Arg 1/0: None
id: 11 coverage: 0 Arg 1/0: None
id: 12 coverage: 0 Arg 1/0: None
id: 13 coverage: 0 Arg 1/0: None
id: 14 coverage: 0 Arg 1/0: None
id: 15 coverage: 0 Arg 1/0: None
id: 16 coverage: 0 Arg 1/0: None
id: 17 coverage: 0 Arg 1/0: None
id: 18 coverage: 0 Arg 1/0: None
id: 19 coverage: 0 Arg 1/0: None
id: 20 coverage: 0 Arg 1/0: None
id: 21 coverage: 0 Arg 1/0: None
id: 22 coverage: 0 Arg 1/0: None
id: 23 coverage: 0 Arg 1/0: None
id: 24 coverage: 0 Arg 1/0: None
id: 25 coverage: 0 Arg 1/0: None
id: 26 cover

In [ ]:
explainer.set_strategy('max_covi', 'universal')
## Define strategy before calling explainer.explain()
explainer.display_explanations()

0.40616693248272195
Number of extensions: 521076
len(max_cov_exts)= 12 / 521076
Covi strategy's coverage: 60
Top 5 covs: [58, 58, 58, 58, 58]
id: 0 coverage: 0 Arg 1/0: None
id: 1 coverage: 5 Arg 1/4: frozenset({'Relationship_Not-in-family', 'Occupation_Exec-managerial'})
id: 2 coverage: 0 Arg 1/0: None
id: 3 coverage: 3 Arg 1/1: frozenset({'Occupation_Machine-op-inspct', 'Age_28.00 < Age <= 37.00'})
id: 4 coverage: 0 Arg 1/0: None
id: 5 coverage: 2 Arg 1/1: frozenset({'Occupation_Farming-fishing', 'Age_28.00 < Age <= 37.00'})
id: 6 coverage: 2 Arg 1/1: frozenset({'Sex_Female', 'Occupation_Protective-serv'})
id: 7 coverage: 3 Arg 1/1: frozenset({'Occupation_Sales', 'Education_Bachelors'})
id: 8 coverage: 2 Arg 1/1: frozenset({'Occupation_?', 'Sex_Male'})
id: 9 coverage: 3 Arg 1/2: frozenset({'Occupation_Prof-specialty', 'Education_Bachelors', 'Workclass_Private'})
id: 10 coverage: 4 Arg 1/1: frozenset({'Occupation_Protective-serv', 'Sex_Male'})
id: 11 coverage: 1 Arg 1/1: frozenset({'A

In [ ]:
anchor_explainer = anchor_tabular.AnchorTabularExplainer(
    dataset.class_names,
    dataset.feature_names,
    dataset.train,
    dataset.categorical_names)
idx = 0
np.random.seed(1)
instance = dataset.train[idx].reshape(1, -1)[0]
print("Anchor")
print([dataset.categorical_names[i][int(r_)] for i, r_ in enumerate(instance)])

print('Prediction: ', anchor_explainer.class_names[c.predict(dataset.train[0].reshape(1, -1))[0]])
exp = anchor_explainer.explain_instance(dataset.train[idx], c.predict, threshold=0.95)
print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

print("Arg:", idx)
print(explainer.explain_instance(idx))

Anchor
['Age > 48.00', 'Self-emp-not-inc', 'Some-college', 'Married-civ-spouse', 'Sales', 'Husband', 'White', 'Male', '2', '0', '40.00 < Hours per week <= 45.00', 'United-States']
Prediction:  b'>50K'
Anchor: Capital Gain = 2
Precision: 1.00
Coverage: 0.07
Arg: 0
id: 0 coverage: 10 Args5/5: {frozenset({'Occupation_Sales', 'Age_Age > 48.00'}), frozenset({'Workclass_Self-emp-not-inc', 'Age_Age > 48.00'}), frozenset({'Workclass_Self-emp-not-inc', 'Occupation_Sales'}), frozenset({'Education_Some-college', 'Workclass_Self-emp-not-inc'}), frozenset({'Education_Some-college', 'Occupation_Sales', 'Country_United-States'})}
None
